In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from JPU import JPU_DeepLab
import tensorflow as tf
import cv2
from tqdm import tqdm
import os
import random
from glob import glob
import pickle
import time
import json
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.python.keras.utils import Sequence
from moviepy.editor import VideoFileClip, ImageSequenceClip
from tensorflow.keras.applications.resnet50 import preprocess_input

/Users/srihari/tf2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/srihari/tf2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/srihari/tf2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/srihari/tf2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

TensorFlow 2.0.0-beta1


In [2]:
h, w = 800, 1600
with open('cityscapes_dict.pkl', 'rb') as f:
    id_to_color = pickle.load(f)['color_map']

In [3]:
model = JPU_DeepLab(h, w, 34)
model.load_weights('top_weights.h5')

171450368/171446536 [==============================] - 95s 1us/step


In [17]:
def pipeline(image, video=True, return_seg=False, fname='', folder=''):
    global b
    alpha = 0.5
    dims = image.shape
    image = cv2.resize(image, (w, h))
    x = image.copy()
    z = model.predict(preprocess_input(np.expand_dims(x, axis=0)))
    z = np.squeeze(z)
    y = np.argmax(z, axis=2)
    
    img_color = image.copy()   
    for i in np.unique(y):
        if i in id_to_color:
            img_color[y==i] = id_to_color[i]
    disp = img_color.copy()
    if video:
        cv2.addWeighted(image, alpha, img_color, 1-alpha, 0, img_color)        
        return img_color
    if return_seg:
        return img_color/255.
    else:
        cv2.addWeighted(image, alpha, img_color, 1-alpha, 0, img_color)      
        plt.figure(figsize=(20, 10))
        out = np.concatenate([image/255, img_color/255, disp/255], axis=1)
        
        plt.imshow(img_color/255.0)
        plt.imshow(out)
        return cv2.imwrite(f'outputs/{folder}/{fname}',  cv2.cvtColor(img_color, cv2.COLOR_RGB2BGR))    

In [18]:
image_dir = '/Users/srihari/datasets/dataset/val_images'
image_list = os.listdir(image_dir)
image_list.sort()
print(f'{len(image_list)} frames found')

500 frames found


In [ ]:
for i in tqdm(range(20)):
    test = load_img(f'{image_dir}/{image_list[i]}')
    test = np.array(test)
    pipeline(test, video=False, fname=f'{i}.png', folder='')

 80%|████████  | 16/20 [04:51<01:12, 18.07s/it]

In [31]:
for image_dir in ['stuttgart_00', 'stuttgart_01', 'stuttgart_02']:
    os.mkdir(f'outputs/{image_dir}')
    image_list = os.listdir(image_dir)
    image_list.sort()
    print(f'{len(image_list)} frames found')
    for i in tqdm(range(len(image_list))):
        try:
            test = plt.imread(f'{image_dir}/{image_list[i]}')
            test = img_to_array(test)
            segmap = pipeline(test, video=False, fname=f'{image_list[i]}', folder=image_dir)
            if segmap == False:
                break
        except Exception as e:
            print(str(e))
    clip = ImageSequenceClip(sorted(glob(f'outputs/{image_dir}/*')), fps=18, load_images=True)
    clip.write_videofile(f'{image_dir}.mp4')

  0%|          | 0/599 [00:00<?, ?it/s]

599 frames found


t:   0%|          | 0/600 [00:00<?, ?it/s, now=None]

Moviepy - Building video stuttgart_00.mp4.
Moviepy - Writing video stuttgart_00.mp4



  0%|          | 0/1100 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

Moviepy - Done !
Moviepy - video ready stuttgart_01.mp4
1200 frames found


100%|██████████| 1200/1200 [11:27<00:00,  1.69it/s]


Moviepy - Building video stuttgart_02.mp4.
Moviepy - Writing video stuttgart_02.mp4



Moviepy - Done !
Moviepy - video ready stuttgart_02.mp4
